# Migrating to Heroku (putting your stuff online)

# Creating our Virtual Environment

### First, we need to organize all the libraries we have installed with pip. When we move our code over to heroku, it won't be able to guess which libraries we need and which ones we don't. We need to have some way to tell our cloud server which dependencies (libraries) to download.

* pip install virtualenv

### We still need to generate a list of requirements for our external server. Let's run the following command in the same directory as our previous command => 
* pip freeze > requirements.txt
* open requirements.txt

### Look at all the files you have installed on your pip! We don't need that many! We are only using a few of those libraries. The rest are used by iPython and other Python applications.

<b>In the top/root level of your django website type in the following command</b> => 
* virtualenv --python=/usr/local/bin/python2.7 .venv
* source .venv/bin/activate

### Wait... what did we just do? Well the answer is kind of complicated. Basically we created a tiny operating system that handles all of our dependencies (libraries) for us. We are going to use this to save our new dependencies to our server. Let's run our command again...

* pip freeze > requirements.txt
* open requirements.txt

### What happend? Where did our dependencies go? Well, since we created a new operating system, we basically need to download all of our libraries again. Copy the following text into your requirements.txt file.

```dj-database-url==0.4.1
dj-static==0.0.6
Django==1.10.3
django-toolbelt==0.0.1
enum==0.4.6
gunicorn==19.6.0
numpy==1.11.2
psycopg2==2.6.2
static3==0.7.0
whitenoise==3.2.2```

### Run the following command to install your new set of libraries
* pip install -r requirements.txt

### Congratulations! You've completed the library exporting.

_<b>Remember: Everything you install from now on will be also installed on your remote server and local environment. Make sure you run the following command before you start working on your project or install new pip libraries.</b>_ => 
* source .venv/bin/activate

_<b>Remember: When you install anything with pip install make sure to run the following command to update your libaries</b>_ => 
* pip freeze > requirements.txt

#### Run =>

* python manage.py runserver

#### If you hit a psycopg2 error, scroll to the bottom of the page

# Setting up our environment

### Now we have all the libraries installed on our computer organized into a file in our root directory

<b>In your settings.py make the following changes or create the variables</b> => 

* ALLOWED_HOSTS = ['0.0.0.0','127.0.0.1','peaceful-badlands-42479.herokuapp.com']
* DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql_psycopg2',
        'NAME': os.environ.get('DATABASE_NAME', ''),
        'HOST': os.environ.get('DATABASE_HOST', ''),
        'USER': os.environ.get('DATABASE_USER', ''),
        'PASSWORD': os.environ.get('DATABASE_PASSWORD', ''),
        'PORT': os.environ.get('DATABASE_PORT', ''),
    }
}
* BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
* STATICFILES_STORAGE = 'whitenoise.django.GzipManifestStaticFilesStorage'
* STATIC_ROOT = os.path.join(BASE_DIR, 'static')

### Remember to save your old DATABASES value. We will need it for later

<b>In your wsgi.py, replace your values with the following values</b> => 

```
import os

from django.core.wsgi import get_wsgi_application
from whitenoise.django import DjangoWhiteNoise

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "classproject.settings")

application = get_wsgi_application()
application = DjangoWhiteNoise(application)
```


### Now we need to make our Procfile. This will tell heroku what to run when we upload the files to our server

* touch Procfile
* open Procfile

<b>put the following text in the file</b> => 

```
web: gunicorn classproject.wsgi
```

### close the file

## Next we have the environment. This is fancy computer talk for a bunch of variables that define things we don't want to show publicly or have to use multiple times over a bunch of programs. Look at what we changed our Databases object to in our settings.py file. Let's define a set of values we can set our new environmental variables to.

### Honcho will help us organize our environmental variables

* pip install honcho

### Let's give our environment some variables!

* touch .env
* open .env

### Insert the following text

```
DATABASE_NAME="mfnyeqfz"
DATABASE_HOST="elmer.db.elephantsql.com"
DATABASE_USER="mfnyeqfz"
DATABASE_PASSWORD="zxzi25vFYOiloQ8ORKUo2-Oog5GgfZc2"
DATABASE_PORT="5432"
```

### Remember to change the variables in the file to your own strings ( They used to be in the settings.py file ) !

* honcho start

# Pushing to Heroku

#### Navigate to:
    http://www.heroku.com/
#### and create an account

#### Add your .env vars to your Heroku console. It should look something like this.
<img src="images/screen01.png" alt="Title"/>

#### Download the cli. This will make things easier for us
    https://devcenter.heroku.com/articles/heroku-command-line

### Making our .gitignore file
#### A .gitignore file tells the remote github server to ignore certain files when they are updated. We don't want to push up our passwords and build files, so it's usually in good practice to make one =>

* touch .gitignore
* open .gitignore

#### Insert the following text into the file

`
venv
*.pyc
staticfiles
.env
`

close .gitignore

### Run the following commands

* heroku login
* heroku create
* heroku buildpacks:set heroku/python
* git add .
* git commit -m "init"

### Now cross your fingers and run the following command =>
* git push heroku master

### If everything worked out well, you should see something like this =>
```
remote: -----> Discovering process types
remote:        Procfile declares types -> web
remote: 
remote: -----> Compressing...
remote:        Done: 84.2M
remote: -----> Launching...
remote:        Released v4
remote:        https://peaceful-badlands-42479.herokuapp.com/ deployed to Heroku
remote: 
remote: Verifying deploy... done.
To https://git.heroku.com/peaceful-badlands-42479.git
 * [new branch]      master -> master
```

### If you do not see this, something went wrong. Don't worry, this is very technically challenging. Look through the documentation available online to see if you can solve the problem from the error message.

### If you are having trouble with psycopg2, sit tight. The answer is complicated...

#### Heroku has the right version of the libraries installed on their servers, but unfortunately, you do not. You need to copy the files over so that your virtual environment can use them. Since we can only use pip to install files, we are left with a problem. We can't use the requirements.txt file to hold our libraries. We have to copy our version of two library files into the right folders.

#### Open a new tab and type in =>

* cd /Applications

#### Find your postgresql application and look for the files below inside. The path should look very similar these paths.  

```
/Applications/Postgres-9.6.0.0.app/Contents/Versions/latest/lib/libcrypto.1.0.0.dylib
/Applications/Postgres-9.6.0.0.app/Contents/Versions/latest/lib/libssl.1.0.0.dylib
```

#### Run the following command with your versions of the files.

* sudo ln -fs /Applications/Postgres.app/Contents/Versions/latest/lib/libcrypto.1.0.0.dylib /usr/local/lib
* sudo ln -fs /Applications/Postgres.app/Contents/Versions/latest/lib/libssl.1.0.0.dylib /usr/local/lib

#### You should now be able to run =>
* python manage.py runserver